In [34]:
import re
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd

In [28]:
data = {'inc_temp': {0: 'warm_30', 1: 'warm_30', 2: 'warm_30', 3: 'warm_30', 4: 'warm_30', 5: 'warm_30', 6: 'warm_30', 7: 'warm_30', 8: 'warm_30', 9: 'warm_30', 10: 'warm_30', 11: 'warm_30', 12: 'warm_30', 13: 'warm_30', 14: 'warm_30', 15: 'warm_30', 16: 'warm_30', 17: 'warm_30', 18: 'warm_30', 19: 'warm_30'}, 'id': {0: '11_8_2018_1_0_A1', 1: '11_8_2018_1_0_A1', 2: '11_8_2018_1_0_A1', 3: '11_8_2018_1_0_A1', 4: '11_8_2018_1_0_A1', 5: '11_8_2018_1_0_A1', 6: '11_8_2018_1_0_A1', 7: '11_8_2018_1_0_A1', 8: '11_8_2018_1_0_A1', 9: '11_8_2018_1_0_A1', 10: '11_8_2018_1_0_A1', 11: '11_8_2018_1_0_A1', 12: '11_8_2018_1_0_A1', 13: '11_8_2018_1_0_A1', 14: '11_8_2018_1_0_A1', 15: '11_8_2018_1_0_A1', 16: '11_8_2018_1_0_A1', 17: '11_8_2018_1_0_A1', 18: '11_8_2018_1_0_A1', 19: '11_8_2018_1_0_A1'}}


In [32]:
df = dd.from_pandas(pd.DataFrame(data), npartitions=2)

In [33]:
df

,inc_temp,id
npartitions=2,,
0,object,object
10,...,...
19,...,...


In [37]:
id_re = re.compile('.*_([A-Z][1-9][0-9]*)$')

In [38]:
def find_well(x):
    assert isinstance(x, str), f"Trying to find well in non-string value {x}"
    match = re.match(id_re, x)
    if match is None:
        raise ValueError(f"Couldn't find well ID in {x}")
    return match.group(1)

find_wellv = np.vectorize(find_well)

In [50]:
wells = find_wellv(df['id'])

In [51]:
wells[0]

'A1'

In [52]:
well_arr = dask.array.from_array(wells)
well_ser = dask.dataframe.from_array(well_arr)
well_df = well_ser.to_frame('well')
well_df.head()

,well
0,A1
1,A1
2,A1
3,A1
4,A1


In [53]:
well_df.npartitions

1

In [54]:
df.npartitions

2

In [55]:
df2 = dask.dataframe.merge(df, well_df)

In [45]:
df2.head()

,inc_temp,id,well
0,warm_30,11_8_2018_1_0_A1,A1
1,warm_30,11_8_2018_1_0_A1,A1
2,warm_30,11_8_2018_1_0_A1,A1
3,warm_30,11_8_2018_1_0_A1,A1
4,warm_30,11_8_2018_1_0_A1,A1


In [57]:
df2.index.compute()

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
            19],
           dtype='int64')

In [49]:
# what I wanted...
df3 = df.copy()
df3['well'] = find_wellv(df3['id'])

TypeError: Column assignment doesn't support type numpy.ndarray

In [48]:
# what I wanted...
df3 = df.copy()
df3['well'] = dask.delayed(find_wellv(df3['id'])).compute()

TypeError: Column assignment doesn't support type numpy.ndarray